# Import libraries

In [ ]:
total = 217

import time
import numpy as np
sd = 30

import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from keras import metrics
from keras.models import Sequential
from keras.layers import Dense, LSTM, GRU
from keras.layers import LeakyReLU, Input, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier

import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.naive_bayes import ComplementNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import CategoricalNB
from sklearn.naive_bayes import MultinomialNB

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

# Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/My Drive/partilhado/

dataset = pd.read_csv('dataset-final.csv', sep=',')

# Split Data



In [ ]:
df = dataset.to_numpy()
x = df[:, 1:]
y = df[:, 0]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = sd)

# Stochastic Gradient Descent

## Grid Search

In [ ]:
parameters = {'alpha': np.logspace(-6, -3, 10), 'penalty' : ['l1','l2']}

st=time.time()
search = GridSearchCV(SGDClassifier(loss = 'log', n_jobs = -1, random_state = sd),
                   parameters, cv = 3, verbose = 11, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train, y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

## Confusion Matrix

In [ ]:
st=time.time()
model = SGDClassifier(loss = 'log', penalty = 'l1', alpha = 0.0001, n_jobs = -1, random_state = sd).fit(x_train,y_train)
print(time.time()-st)

yt = model.predict(x_test)

m = confusion_matrix(y_test, yt, normalize = 'true')
tick = sorted(set(y_test))
fig, ax = plt.subplots(figsize=(30,15))
conf = sns.heatmap(m, xticklabels = tick, yticklabels = tick, cmap=plt.cm.Blues)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
conf.xaxis.tick_top()
conf.xaxis.set_label_position('top')

#Logistic Regression

## Grid Search

In [ ]:
parameters = {'C' : np.logspace(-1, 3, 10), 'solver' : ['lbfgs']}

st=time.time()
search = GridSearchCV(LogisticRegression(multi_class = 'multinomial', n_jobs = -1, random_state = sd),
                   parameters, cv = 3, verbose = 11, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train,y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

In [ ]:
parameters = {'C' : np.logspace(-1, 3, 10), 'solver' : ['sag']}

st=time.time()
search = GridSearchCV(LogisticRegression(max_iter = 400, multi_class = 'multinomial', n_jobs = -1, random_state = sd),
                   parameters, cv = 3, verbose = 11, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train,y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

## Confusion Matrix

In [ ]:
st=time.time()
model = LogisticRegression(multi_class = 'multinomial', n_jobs = -1, random_state = sd).fit(x_train,y_train)
print(time.time()-st)

yt = model.predict(x_test)

m = confusion_matrix(y_test, yt, normalize = 'true')
tick = sorted(set(y_test))
fig, ax = plt.subplots(figsize=(30,15))
conf = sns.heatmap(m, xticklabels = tick, yticklabels = tick, cmap=plt.cm.Blues)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
conf.xaxis.tick_top()
conf.xaxis.set_label_position('top')

# Naive Bayes

##BernoulliNB

In [ ]:
param = np.logspace(-4, 3, 50)
parameters = {'alpha': param}

st=time.time()
search = GridSearchCV(BernoulliNB(), parameters, cv = 3, return_train_score = True,
                      verbose = 10, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train, y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

results = search.cv_results_
trn_scores_mean = results['mean_train_score']
trn_scores_std = results['std_train_score']
val_scores_mean = results['mean_test_score']
val_scores_std = results['std_test_score']


plt.title("Validation Curve with BernoulliNB")
plt.xlabel(r"$\alpha$")
plt.ylabel("F1-score")
plt.semilogx(param, trn_scores_mean, label="Training score", color="darkorange", lw=2)
plt.fill_between(param, trn_scores_mean - trn_scores_std,
                 trn_scores_mean + trn_scores_std, alpha=0.2,
                 color="darkorange", lw=2)
plt.semilogx(param, val_scores_mean, label="Cross-validation score", color="navy", lw=2)
plt.fill_between(param, val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std, alpha=0.2,
                 color="navy", lw=2)
plt.legend(loc="best")
plt.show()

##ComplementNB

In [ ]:
param = np.logspace(0, 3, 50)
parameters = {'alpha': param}

st=time.time()
search = GridSearchCV(ComplementNB(), parameters, cv = 3, return_train_score = True,
                      verbose = 10, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train, y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

results = search.cv_results_
trn_scores_mean = results['mean_train_score']
trn_scores_std = results['std_train_score']
val_scores_mean = results['mean_test_score']
val_scores_std = results['std_test_score']

plt.title("Validation Curve with ComplementNB")
plt.xlabel(r"$\alpha$")
plt.ylabel("F1-score")
plt.semilogx(param, trn_scores_mean, label="Training score", color="darkorange", lw=2)
plt.fill_between(param, trn_scores_mean - trn_scores_std,
                 trn_scores_mean + trn_scores_std, alpha=0.2,
                 color="darkorange", lw=2)
plt.semilogx(param, val_scores_mean, label="Cross-validation score", color="navy", lw=2)
plt.fill_between(param, val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std, alpha=0.2,
                 color="navy", lw=2)
plt.legend(loc="best")
plt.show()

## Confusion Matrix

In [ ]:
st=time.time()
model = BernoulliNB(alpha = 100).fit(x_train,y_train)
#model = ComplementNB(alpha = 184).fit(x_train,y_train)
print(time.time()-st)

yt = model.predict(x_test)

m = confusion_matrix(y_test, yt, normalize = 'true')
tick = sorted(set(y_test))
fig, ax = plt.subplots(figsize=(30,15))
conf = sns.heatmap(m, xticklabels = tick, yticklabels = tick, cmap=plt.cm.Blues)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
conf.xaxis.tick_top()
conf.xaxis.set_label_position('top')

# Random Forest

## Grid Search

In [ ]:
parameters = {'n_estimators' : list(range(50,150,25)), 'criterion' : ['gini','entropy'],
              'bootstrap' : [True,False], 'max_features' : ['auto', 'sqrt', 'log2']}

st=time.time()
search = GridSearchCV(RandomForestClassifier(n_jobs = -1, random_state = sd),
                   parameters, cv = 3, verbose = 10, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train, y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

## Validation Curves

In [ ]:
param = list(range(2,52,2))
parameters = {'min_samples_split' : param}

st=time.time()
search = GridSearchCV(RandomForestClassifier(n_estimators = 75, n_jobs = -1, random_state = sd),
                      parameters, cv = 3, return_train_score = True,
                      verbose = 10, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train, y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

results = search.cv_results_
trn_scores_mean = results['mean_train_score']
trn_scores_std = results['std_train_score']
val_scores_mean = results['mean_test_score']
val_scores_std = results['std_test_score']


plt.title('Validation Curve for min_samples_split')
plt.xlabel('min_samples_split')
plt.ylabel('F1-score')
plt.semilogx(param, trn_scores_mean, label='Training score', color='darkorange', lw=2)
plt.fill_between(param, trn_scores_mean - trn_scores_std,
                 trn_scores_mean + trn_scores_std, alpha=0.2,
                 color='darkorange', lw=2)
plt.semilogx(param, val_scores_mean, label='Cross-validation score', color='navy', lw=2)
plt.fill_between(param, val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std, alpha=0.2,
                 color='navy', lw=2)
plt.legend(loc='best')
plt.show()

In [ ]:
param = list(range(1,51,2))
parameters = {'min_samples_leaf' : param}

st=time.time()
search = GridSearchCV(RandomForestClassifier(min_samples_split = 8, n_estimators = 75, n_jobs = -1, random_state = sd),
                      parameters, cv = 3, return_train_score = True,
                      verbose = 10, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train, y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

results = search.cv_results_
trn_scores_mean = results['mean_train_score']
trn_scores_std = results['std_train_score']
val_scores_mean = results['mean_test_score']
val_scores_std = results['std_test_score']


plt.title('Validation Curve for min_samples_leaf')
plt.xlabel('min_samples_leaf')
plt.ylabel('F1-score')
plt.semilogx(param, trn_scores_mean, label='Training score', color='darkorange', lw=2)
plt.fill_between(param, trn_scores_mean - trn_scores_std,
                 trn_scores_mean + trn_scores_std, alpha=0.2,
                 color='darkorange', lw=2)
plt.semilogx(param, val_scores_mean, label='Cross-validation score', color='navy', lw=2)
plt.fill_between(param, val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std, alpha=0.2,
                 color='navy', lw=2)
plt.legend(loc='best')
plt.show()

## Confusion Matrix

In [ ]:
st=time.time()
model = RandomForestClassifier(n_estimators = 75, min_samples_split = 8, n_jobs = -1, random_state = sd).fit(x_train,y_train)
print(time.time()-st)

yt = model.predict(x_test)

m = confusion_matrix(y_test, yt, normalize = 'true')
tick = sorted(set(y_test))
fig, ax = plt.subplots(figsize=(30,15))
conf = sns.heatmap(m, xticklabels = tick, yticklabels = tick, cmap=plt.cm.Blues)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
conf.xaxis.tick_top()
conf.xaxis.set_label_position('top')

# MLP

## Grid Search

In [ ]:
parameters = {'hidden_layer_sizes' : list(range(50,100,25)), 'activation' : ['tanh','relu']}

st=time.time()
search = GridSearchCV(MLPClassifier(max_iter = 400, hidden_layer_sizes = (75), random_state = sd),
                      parameters, cv = 3, verbose = 10, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train, y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

## Validation Curve

In [ ]:
param = np.logspace(-12,-8,20)
parameters = {'alpha' : param}

st=time.time()
search = GridSearchCV(MLPClassifier(hidden_layer_sizes = (75), early_stopping = True, random_state = sd),
                      parameters, cv = 3, return_train_score = True,
                      verbose = 10, n_jobs = -1, scoring = 'f1_weighted')
search.fit(x_train, y_train)
print(time.time()-st)

print(search.best_params_)
print(search.best_score_)
print(search.cv_results_['std_test_score'][search.best_index_])

results = search.cv_results_
trn_scores_mean = results['mean_train_score']
trn_scores_std = results['std_train_score']
val_scores_mean = results['mean_test_score']
val_scores_std = results['std_test_score']


plt.title('Validation Curve with MLP')
plt.xlabel(r'$\alpha$')
plt.ylabel('F1-score')
plt.semilogx(param, trn_scores_mean, label='Training score', color='darkorange', lw=2)
plt.fill_between(param, trn_scores_mean - trn_scores_std,
                 trn_scores_mean + trn_scores_std, alpha=0.2,
                 color='darkorange', lw=2)
plt.semilogx(param, val_scores_mean, label='Cross-validation score', color='navy', lw=2)
plt.fill_between(param, val_scores_mean - val_scores_std,
                 val_scores_mean + val_scores_std, alpha=0.2,
                 color='navy', lw=2)
plt.legend(loc='best')
plt.show()

## Confusion Matrix

In [ ]:
st=time.time()
model = MLPClassifier(hidden_layer_sizes = (75), alpha = 10e-10, early_stopping = True, random_state = sd).fit(x_train,y_train)
print(time.time()-st)

yt = model.predict(x_test)

m = confusion_matrix(y_test, yt, normalize = 'true')
tick = sorted(set(y_test))
fig, ax = plt.subplots(figsize=(30,15))
conf = sns.heatmap(m, xticklabels = tick, yticklabels = tick, cmap=plt.cm.Blues)
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
conf.xaxis.tick_top()
conf.xaxis.set_label_position('top')

# Scores Comparison

In [ ]:
model = SGDClassifier(loss = 'log', penalty = 'l1', alpha = 0.0001, n_jobs = -1, random_state = sd).fit(x_train,y_train)
yv = model.predict(x_test)
c1 = classification_report(y_test, yv, digits=3, output_dict = True)

model = MLPClassifier(hidden_layer_sizes = (75), alpha = 10e-10, early_stopping = True, random_state = sd).fit(x_train,y_train)
yv = model.predict(x_test)
c2 = classification_report(y_test, yv, digits=3, output_dict = True)

## Classification reports difference

In [ ]:
def rec_diff(d1,d2):
  diff = dict()
  for k,v1 in d1.items():
    if isinstance(v1, dict):
      diff[k] = rec_diff(v1, d2[k])
    else:
      if k != 'support':
        diff[k] = v1 - d2[k]
      else:
        diff[k] = v1
  return diff

# mlp - sgd
c3 = rec_diff(c2, c1)

In [ ]:
for k in c3:
  if k.isnumeric() and (c3[k]['precision'] != 0 or c3[k]['recall'] != 0):
    print(k,c3[k],'\n')
  elif k.isnumeric() == False:
    print(k,c3[k],'\n')

# Keras NN

In [ ]:
from keras import backend as K

ytrain = to_categorical(y_train, num_classes = total)
yval = to_categorical(y_test, num_classes = total)

def f1_weighted(true, pred):
  predLabels = K.argmax(pred, axis=-1)
  pred = K.one_hot(predLabels, total) 

  actual_positives = K.sum(true, axis=0)       # = TP + FN
  pred_positives = K.sum(pred, axis=0)         # = TP + FP
  true_positives = K.sum(true * pred, axis=0)  # = TP

  precision = (true_positives + K.epsilon()) / (pred_positives + K.epsilon()) 
  recall = (true_positives + K.epsilon()) / (actual_positives + K.epsilon()) 
  #both = 1 if ground_positives == 0 or pred_positives == 0

  f1 = 2 * (precision * recall) / (precision + recall)

  weighted_f1 = f1 * actual_positives / K.sum(actual_positives)
  weighted_f1 = K.sum(weighted_f1)

  return weighted_f1

## Cross Validation

In [ ]:
kfold = StratifiedKFold(n_splits = 10)
cvscores = []

st=time.time()
for train, val in kfold.split(x_train, y_train):
	# create model
	model = Sequential()
	model.add(Dense(300, input_dim = 565, activation = 'relu'))
	model.add(Dropout(0.2))
	model.add(LeakyReLU())
	model.add(Dropout(0.3))
	model.add(Dense(total, activation = 'softmax'))
 
	model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [f1_weighted])
	model.fit(x_train[train], ytrain[train], batch_size = 1000, epochs = 150, verbose = False)
	
	# evaluate the model
	scores = model.evaluate(x_train[val], ytrain[val], verbose = False)
	print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
	cvscores.append(scores[1]*100)
print(time.time()-st)
 
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

In [ ]:
from keras.utils.vis_utils import plot_model

clf = Sequential()
clf.add(Dense(300, input_dim = 565, activation = 'relu'))
clf.add(Dropout(0.2))
clf.add(LeakyReLU())
clf.add(Dropout(0.3))
clf.add(Dense(total, activation = 'softmax'))

clf.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [f1_weighted])

plot_model(clf, to_file='model_nn.png', show_shapes=True, show_layer_names=True)

In [ ]:
st=time.time()
model = Sequential()
model.add(Dense(300, input_dim = 565, activation = 'relu'))
model.add(Dropout(0.2))
model.add(LeakyReLU())
model.add(Dropout(0.3))
model.add(Dense(total, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = [f1_weighted])
model.fit(x_train, ytrain, batch_size = 1000, epochs = 150, verbose = False)
print(time.time()-st)

yt = np.argmax(model.predict(x_train), axis=-1)
print("Accuracy:", accuracy_score(y_train, yt))
print("F1-score:", f1_score(y_train, yt, average = 'weighted'))

yv = np.argmax(model.predict(x_test), axis=-1)
print("Accuracy:", accuracy_score(y_test, yv))
print("F1-score:", f1_score(y_test, yv, average = 'weighted'))

## Confusion Matrix

In [ ]:
m = confusion_matrix(y_test, yv, normalize = 'true')
tick = sorted(set(y_train))
fig, ax = plt.subplots(figsize = (40,20))
conf = sns.heatmap(m, cmap=plt.cm.Blues)
plt.ylabel('True Label', fontsize = 25)
plt.xlabel('Predicted Label', fontsize = 25)
conf.set_xticklabels(tick, fontsize = 12)
conf.set_yticklabels(tick, fontsize = 12, rotation = 0)
conf.xaxis.tick_top()
conf.xaxis.set_label_position('top')

# Box Plot

In [ ]:
def evaluate_model(x, y, model):
	cv = StratifiedKFold(n_splits = 3)
	scores = cross_val_score(model, x, y, scoring = 'f1_weighted', cv = cv, n_jobs = -1)
	return scores

models, names = list(), list()

models.append(BernoulliNB(alpha = 100))
names.append('BernoulliNB')

models.append(ComplementNB(alpha = 184))
names.append('ComplementNB')

models.append(SGDClassifier(loss = 'log', penalty = 'l1', alpha = 0.0001, n_jobs = -1, random_state = sd))
names.append('SGD')
	
models.append(LogisticRegression(multi_class = 'multinomial', n_jobs = -1, random_state = sd))
names.append('LR')

models.append(RandomForestClassifier(n_estimators = 75, min_samples_split = 8, n_jobs = -1, random_state = sd))
names.append('RF')

models.append(MLPClassifier(hidden_layer_sizes = (75), alpha = 10e-10, early_stopping = True, random_state = sd))
names.append('MLP')

results = list()
for i in range(len(models)):
	scores = evaluate_model(x_train, y_train, models[i])
	results.append(scores)

	print('%s: %.3f (%.3f)' % (names[i], np.mean(scores), np.std(scores)))

In [ ]:
plt.figure(figsize = (25,15))
plt.boxplot(results, labels = names, showmeans = True)
plt.xticks(fontsize = 15)
plt.yticks(fontsize = 15)
plt.xlabel('Classifiers', fontsize = 20)
plt.ylabel('Weighted F1-score', fontsize = 20)
plt.show()



---



# Resampling

In [ ]:
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

## Under-Sampling

In [ ]:
c = Counter(y_train)
c[153] = 10000

rus = RandomUnderSampler(random_state = sd, sampling_strategy = c)
x_rus, y_rus = rus.fit_resample(x_train, y_train)
print(sorted(Counter(y_rus).items()))

In [ ]:
st=time.time()
model = LogisticRegression(multi_class = 'multinomial', n_jobs = -1, random_state = sd).fit(x_rus,y_rus)
print(time.time()-st)

yt = model.predict(x_rus)
print("Accuracy:", accuracy_score(y_rus, yt))
print("F1-score:", f1_score(y_rus, yt, average='weighted'))
        
yv = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, yv))
print("F1-score:", f1_score(y_test, yv, average='weighted'))
#print(classification_report(y_test, yv, digits=3))

## Over-Sampling

In [ ]:
c = Counter(y_train)
for item in c:
  if c[item] < 100:
    c[item] = c[item] + 100

ros = RandomOverSampler(random_state = sd, sampling_strategy = c)
x_ros, y_ros = ros.fit_resample(x_train, y_train)
print(sorted(Counter(y_ros).items()))

In [ ]:
st=time.time()
model = LogisticRegression(multi_class = 'multinomial', n_jobs = -1, random_state = sd).fit(x_ros,y_ros)
print(time.time()-st)

yt = model.predict(x_ros)
print("Accuracy:", accuracy_score(y_ros, yt))
print("F1-score:", f1_score(y_ros, yt, average='weighted'))
        
yv = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, yv))
print("F1-score:", f1_score(y_test, yv, average='weighted'))
#print(classification_report(y_test, yv, digits=3))

## Combination

In [ ]:
c = Counter(y_train)
for item in c:
  if c[item] < 100:
    c[item] = c[item] + 100

ros = RandomOverSampler(random_state = sd, sampling_strategy = c)
x_ros, y_ros = ros.fit_resample(x_train, y_train)

c[153] = 10000

rus = RandomUnderSampler(random_state = sd, sampling_strategy = c)
x_rus, y_rus = rus.fit_resample(x_ros, y_ros)

In [ ]:
st=time.time()
model = LogisticRegression(multi_class = 'multinomial', n_jobs = -1, random_state = sd).fit(x_rus,y_rus)
print(time.time()-st)

yt = model.predict(x_rus)
print("Accuracy:", accuracy_score(y_rus, yt))
print("F1-score:", f1_score(y_rus, yt, average='weighted'))
        
yv = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, yv))
print("F1-score:", f1_score(y_test, yv, average='weighted'))
#print(classification_report(y_test, yv, digits=3))

## Balance Data

### Remove noise

In [ ]:
data = dataset.copy()
ycounts = pd.Series(data['LABEL']).value_counts()
noise = ycounts[ycounts < 10].index
data = data[~data['LABEL'].isin(noise)]

### Group minority classes

In [ ]:
#data = dataset.copy()
ycounts = pd.Series(data['LABEL']).value_counts()
noise = ycounts[ycounts < 50].index
data['LABEL'].replace(noise, total, inplace=True)

### Model

In [ ]:
df = data.to_numpy()
x = df[:, 1:]
y = df[:, 0]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, stratify = y, random_state = sd)

In [ ]:
st=time.time()
model = LogisticRegression(multi_class = 'multinomial', n_jobs = -1, random_state = sd).fit(x_train,y_train)
print(time.time()-st)

yt = model.predict(x_train)
print("Accuracy:", accuracy_score(y_train, yt))
print("F1-score:", f1_score(y_train, yt, average='weighted'))
        
yv = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, yv))
print("F1-score:", f1_score(y_test, yv, average='weighted'))
#print(classification_report(y_test, yv, digits=3))